### Import libraries

In [6]:
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from sklearn.preprocessing import LabelEncoder

print('Libraries imported.')

Libraries imported.


### Import saved neighborhood data

In [144]:
neighborhood_data = pd.read_csv('Data/neighborhood_data.csv')
neighborhood_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Get latitude and longitude of Toronto

In [3]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent='totonto')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


### Label Encode Borough

In [7]:
lb = LabelEncoder()
neighborhood_data['Encoded Borough'] = lb.fit_transform(neighborhood_data['Borough'])
neighborhood_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Encoded Borough
0,M3A,North York,Parkwoods,43.753259,-79.329656,6
1,M4A,North York,Victoria Village,43.725882,-79.315572,6
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,6
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1


### Plot Neighborhoods on Toronto Map

In [15]:
toronto_map_all = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the boroughs
borough_count = len(neighborhood_data['Encoded Borough'].unique())
x = np.arange(borough_count)
ys = [i + x + (i*x)**2 for i in range(borough_count)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, long, neighbour, borough_code, borough in zip(neighborhood_data['Latitude'], neighborhood_data['Longitude'], neighborhood_data['Neighborhood'], neighborhood_data['Encoded Borough'], neighborhood_data['Borough']):
    label = folium.Popup("{}: {}".format(borough, neighbour), parse_html=True)
    folium.CircleMarker(
    [lat, long],
    radius=5,
    popup=label,
    color=rainbow[borough_code-1],
    fill=True,
    fill_color=rainbow[borough_code-1],
    parse_html=True).add_to(toronto_map_all)
toronto_map_all

### Setup credentials for foursquare app

In [16]:
CLIENT_ID = 'RK0UKQG2Q4P42VBMT5DCZZMJTZWK2KWMP5BNQWC4QDPIXNFL' # your Foursquare ID
CLIENT_SECRET = 'WPNKKWYXHP4XOG30U3FQ0MUBYUO2L5ARLDXYSIKJNKKW33LK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RK0UKQG2Q4P42VBMT5DCZZMJTZWK2KWMP5BNQWC4QDPIXNFL
CLIENT_SECRET:WPNKKWYXHP4XOG30U3FQ0MUBYUO2L5ARLDXYSIKJNKKW33LK


### Get venues near Neighborhood

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, end="; ")
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
neighbour_venues = getNearbyVenues(neighborhood_data['Neighborhood'], neighborhood_data['Latitude'], neighborhood_data['Longitude'])

Parkwoods; Victoria Village; Regent Park, Harbourfront; Lawrence Manor, Lawrence Heights; Queen's Park, Ontario Provincial Government; Islington Avenue, Humber Valley Village; Malvern, Rouge; Don Mills; Parkview Hill, Woodbine Gardens; Garden District, Ryerson; Glencairn; West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale; Rouge Hill, Port Union, Highland Creek; Don Mills; Woodbine Heights; St. James Town; Humewood-Cedarvale; Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood; Guildwood, Morningside, West Hill; The Beaches; Berczy Park; Caledonia-Fairbanks; Woburn; Leaside; Central Bay Street; Christie; Cedarbrae; Hillcrest Village; Bathurst Manor, Wilson Heights, Downsview North; Thorncliffe Park; Richmond, Adelaide, King; Dufferin, Dovercourt Village; Scarborough Village; Fairview, Henry Farm, Oriole; Northwood Park, York University; East Toronto, Broadview North (Old East York); Harbourfront East, Union Station, Toronto Islands; Little Portugal, Trinit

In [26]:
neighborhood_venues = neighbour_venues
neighborhood_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


### Add borough column to venues data

In [30]:
neighborhood_venues = pd.merge(neighborhood_venues, neighborhood_data[['Borough', 'Neighborhood']])
neighborhood_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park,North York
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop,North York
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping,North York
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena,North York
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant,North York


### Number of venues in each neighbourhood

In [38]:
venues_count_neighbour = neighborhood_venues.groupby('Neighborhood').count().iloc[:, :1]
venues_count_neighbour.columns = ['Count']
venues_count_neighbour.sort_values('Count', ascending=False)

,Count
Neighborhood,
"Garden District, Ryerson",100
"Commerce Court, Victoria Hotel",100
"Toronto Dominion Centre, Design Exchange",100
"Harbourfront East, Union Station, Toronto Islands",100
"First Canadian Place, Underground city",100
...,...
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",1
"Malvern, Rouge",1
"Rouge Hill, Port Union, Highland Creek",1


# Anaysis on borroughs which contains 'Toronto' in its name

### Get venues of boroughs that contain 'Toronto' in its name

In [46]:
toronto_neighborhood = neighborhood_venues[neighborhood_venues['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_neighborhood.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,Downtown Toronto
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,Downtown Toronto
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,Downtown Toronto
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,Downtown Toronto
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub,Downtown Toronto


### Get count of venues in neighborhood

In [51]:
x = toronto_neighborhood.groupby(['Borough', 'Neighborhood']).count().iloc[:, :1]
x.columns = ['Count']
x

Count
Borough          Neighborhood                                             
Central Toronto  Davisville                                             33
                 Davisville North                                        9
                 Forest Hill North & West, Forest Hill Road Park         6
                 Lawrence Park                                           4
                 Moore Park, Summerhill East                             3
                 North Toronto West, Lawrence Park                      20
                 Roselawn                                                1
                 Summerhill West, Rathnelly, South Hill, Forest ...     16
                 The Annex, North Midtown, Yorkville                    21
Downtown Toronto Berczy Park                                            56
                 CN Tower, King and Spadina, Railway Lands, Harb...     16
                 Central Bay Street                                     64
                 Christie                                               17
                 Church and Wellesley                                   75
                 Commerce Court, Victoria Hotel                        100
                 First Canadian Place, Underground city                100
                 Garden District, Ryerson                              100
                 Harbourfront East, Union Station, Toronto Islands     100
                 Kensington Market, Chinatown, Grange Park              62
                 Queen's Park, Ontario Provincial Government            34
                 Regent Park, Harbourfront                              44
                 Richmond, Adelaide, King                               92
                 Rosedale                                                4
                 St. James Town                                         78
                 St. James Town, Cabbagetown                            46
                 Stn A PO Boxes                                         96
                 Toronto Dominion Centre, Design Exchange              100
                 University of Toronto, Harbord                         34
East Toronto     Business reply mail Processing Centre, South Ce...     16
                 India Bazaar, The Beaches West                         24
                 Studio District                                        40
                 The Beaches                                             5
                 The Danforth West, Riverdale                           43
West Toronto     Brockton, Parkdale Village, Exhibition Place           24
                 Dufferin, Dovercourt Village                           16
                 High Park, The Junction South                          25
                 Little Portugal, Trinity                               43
                 Parkdale, Roncesvalles                                 15
                 Runnymede, Swansea                                     38

### onehot encoding on category

In [88]:
onehot = pd.get_dummies(toronto_neighborhood['Venue Category'])
onehot.insert(0, 'Borough', toronto_neighborhood['Borough'])
onehot.insert(1, 'Neighborhoods', toronto_neighborhood['Neighborhood'])
onehot.head()

,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Category mean

In [94]:
toronto_grouped = onehot.groupby(['Borough', 'Neighborhoods']).mean().reset_index()
toronto_grouped.head()

,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Central Toronto,Davisville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.030303,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Central Toronto,Davisville North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Central Toronto,"Moore Park, Summerhill East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Get top 10 most common venues in each neighborhood

In [115]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods', 'Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [118]:
neighborhood_venues_sorted = pd.DataFrame(columns=columns)

def return_most_common_venues(row, num_top_venues):
    
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

neighborhood_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhood_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

In [120]:
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [122]:
neighborhood_venues_sorted.head()

,Neighborhoods,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Central Toronto,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gym,Diner,Indian Restaurant
1,Davisville North,Central Toronto,Park,Pizza Place,Breakfast Spot,Sandwich Place,Department Store,Food & Drink Shop,Convenience Store,Hotel,Gym,Concert Hall
2,"Forest Hill North & West, Forest Hill Road Park",Central Toronto,Trail,Park,Bus Line,Jewelry Store,Sushi Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
3,Lawrence Park,Central Toronto,Park,Bus Line,Swim School,Lawyer,Colombian Restaurant,College Gym,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
4,"Moore Park, Summerhill East",Central Toronto,Park,Restaurant,Playground,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


# Clustering neighborhoods in Boroughs which contains 'Toronto' in their name

In [173]:
n_cluster = 4
toronto_clusturing = toronto_grouped.drop(['Neighborhoods', 'Borough'], axis=1)
toronto_clusturing.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.030303,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [174]:
kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(toronto_clusturing)

In [175]:
neighborhood_venues_sorted.drop('Cluster', axis=1, inplace=True)

In [176]:
neighborhood_venues_sorted.insert(2, 'Cluster', kmeans.labels_)
neighborhood_venues_sorted.head()

,Neighborhoods,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gym,Diner,Indian Restaurant
1,Davisville North,Central Toronto,0,Park,Pizza Place,Breakfast Spot,Sandwich Place,Department Store,Food & Drink Shop,Convenience Store,Hotel,Gym,Concert Hall
2,"Forest Hill North & West, Forest Hill Road Park",Central Toronto,2,Trail,Park,Bus Line,Jewelry Store,Sushi Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
3,Lawrence Park,Central Toronto,2,Park,Bus Line,Swim School,Lawyer,Colombian Restaurant,College Gym,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
4,"Moore Park, Summerhill East",Central Toronto,2,Park,Restaurant,Playground,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


### Merge neighborhood_data and neighborhood_venues_sorted for Latitude and Longitude

In [177]:
neighborhood_data = neighborhood_data.rename(columns={'Neighborhood': 'Neighborhoods'})
merged_data_clustered = pd.merge(neighborhood_venues_sorted, neighborhood_data[['Neighborhoods', 'Latitude', 'Longitude']], on='Neighborhoods')
merged_data_clustered.head()

,Neighborhoods,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,Davisville,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gym,Diner,Indian Restaurant,43.704324,-79.388790
1,Davisville North,Central Toronto,0,Park,Pizza Place,Breakfast Spot,Sandwich Place,Department Store,Food & Drink Shop,Convenience Store,Hotel,Gym,Concert Hall,43.712751,-79.390197
2,"Forest Hill North & West, Forest Hill Road Park",Central Toronto,2,Trail,Park,Bus Line,Jewelry Store,Sushi Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,43.696948,-79.411307
3,Lawrence Park,Central Toronto,2,Park,Bus Line,Swim School,Lawyer,Colombian Restaurant,College Gym,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,43.728020,-79.388790
4,"Moore Park, Summerhill East",Central Toronto,2,Park,Restaurant,Playground,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,43.689574,-79.383160


### Clusters on Toronto map

In [178]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(n_cluster)
ys = [i + x + (i*x)**2 for i in range(n_cluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

marker_color = []
for lat, long, name, cluster in zip(merged_data_clustered['Latitude'], merged_data_clustered['Longitude'], merged_data_clustered['Neighborhoods'], merged_data_clustered['Cluster']):
    label = folium.Popup("{} : Cluster {}".format(name, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, long],
        popup=label,
        color = rainbow[cluster-1],
        fill=True,
        fill_color = rainbow[cluster-1],
        fill_opacity=0.6,
        parse_html=True,
        radius=5
    ).add_to(toronto_map)
toronto_map

### Neighborhood in each cluster

In [181]:
for i in np.arange(n_cluster):
    print("Cluster {}".format(i))
    display(merged_data_clustered[merged_data_clustered['Cluster'] == i])
    print("\n\n\n\n")

Cluster 0


,Neighborhoods,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,Davisville,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gym,Diner,Indian Restaurant,43.704324,-79.388790
1,Davisville North,Central Toronto,0,Park,Pizza Place,Breakfast Spot,Sandwich Place,Department Store,Food & Drink Shop,Convenience Store,Hotel,Gym,Concert Hall,43.712751,-79.390197
5,"North Toronto West, Lawrence Park",Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Fast Food Restaurant,Rental Car Location,Mexican Restaurant,Shoe Store,Restaurant,Café,Sporting Goods Shop,43.715383,-79.405678
7,"Summerhill West, Rathnelly, South Hill, Forest...",Central Toronto,0,Pub,Coffee Shop,Supermarket,Vietnamese Restaurant,Sushi Restaurant,Light Rail Station,Bank,Liquor Store,American Restaurant,Restaurant,43.686412,-79.400049
8,"The Annex, North Midtown, Yorkville",Central Toronto,0,Café,Sandwich Place,Coffee Shop,Pizza Place,Pub,Middle Eastern Restaurant,BBQ Joint,Indian Restaurant,History Museum,Burger Joint,43.672710,-79.405678
9,Berczy Park,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Cheese Shop,Seafood Restaurant,Café,Restaurant,Greek Restaurant,Irish Pub,43.644771,-79.373306
10,"CN Tower, King and Spadina, Railway Lands, Har...",Downtown Toronto,0,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boutique,Boat or Ferry,Rental Car Location,43.628947,-79.394420
11,Central Bay Street,Downtown Toronto,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Thai Restaurant,43.657952,-79.387383
12,Christie,Downtown Toronto,0,Grocery Store,Café,Park,Diner,Candy Store,Baby Store,Restaurant,Athletics & Sports,Italian Restaurant,Nightclub,43.669542,-79.422564
13,Church and Wellesley,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Bubble Tea Shop,Burger Joint,Mediterranean Restaurant,Pub,43.665860,-79.383160







Cluster 1


,Neighborhoods,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
6,Roselawn,Central Toronto,1,Garden,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,43.711695,-79.416936







Cluster 2


,Neighborhoods,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
2,"Forest Hill North & West, Forest Hill Road Park",Central Toronto,2,Trail,Park,Bus Line,Jewelry Store,Sushi Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,43.696948,-79.411307
3,Lawrence Park,Central Toronto,2,Park,Bus Line,Swim School,Lawyer,Colombian Restaurant,College Gym,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,43.728020,-79.388790
4,"Moore Park, Summerhill East",Central Toronto,2,Park,Restaurant,Playground,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,43.689574,-79.383160
22,Rosedale,Downtown Toronto,2,Park,Playground,Trail,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,43.679563,-79.377529







Cluster 3


,Neighborhoods,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
31,The Beaches,East Toronto,3,Health Food Store,Neighborhood,Asian Restaurant,Pub,Trail,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Yoga Studio,43.676357,-79.293031


### Observations

It can be observed that most of the neighborhoods fall in Cluster 0, Cluster 0 is business spot because most of the venues situated in this Cluster are money making venues. Most popular spots in Cluster 0 are restaurants, among which coffee shops and cafes are most popular spots in these areas, these neighborhoods may be cosmopolitan areas of the city. Cluster 2 contains neighborhoods which are more  close to nature, most popular spots in these neighborhood are parks and trails, these neighborhoods are excellent place for chindrens and seniour citizen as it contains a lot of parks and playground. 